## Specification DDE Metadata generator

This script ingests information from Sahar's automatically pushed list of recently updated files to create changes in the appropriate tables that are used by the DDE.

In [21]:
import os
import json
import pandas as pd
import time
import requests

In [83]:
def convert_to_raw(githuburl): ## Converts a github url to a raw github url
    githubrawurl = githuburl.replace('github.com','raw.githubusercontent.com').replace('blob/','').replace('tree/','')
    return githubrawurl

def load_parent_source():
    parent_source_url = 'https://github.com/BioSchemas/bioschemas.github.io/blob/profile-auto-generation/_data/metadata_mapping.csv'
    parent_source_df = pd.read_csv(convert_to_raw(parent_source_url), header=0,usecols = ['profile','TypeParent','ProfileParent'])
    return parent_source_df

def lookup_parent(parent_source_options,classname,spectype):
    parent_choices = parent_source_options.loc[parent_source_options['profile']==classname]
    if spectype == 'Profile':
        parent = parent_choices.iloc[0]['ProfileParent']
    else:
        parent = parent_choices.iloc[0]['TypeParent']
    return parent

def parse_info_from_json(url,parent_source_options):
    r = requests.get(convert_to_raw(url))
    tmpjson = json.loads(r.text)
    speclass =  [x for x in tmpjson['@graph'] if x["@type"]=="rdfs:Class"]
    speclassid = speclass[0]['@id'].split(':')
    namespace = speclassid[0]
    classname = speclassid[1]
    if '$validation' in list(speclass[0].keys()):
        spectype = 'Profile'
    else:
        spectype = 'Type'
    version = parse_version_from_url(url,classname)
    parent = lookup_parent(parent_source_options,classname,spectype)
    tmpdict = {'namespace':namespace,'name':classname,'subClassOf':parent,'type':spectype,'version':version,'url':url}
    return tmpdict

def generate_base_update_table(script_path):
    updated_specs_folder = os.path.join(script_path,'latest-updated-profiles')
    latest_updates = os.listdir(updated_specs_folder)
    parent_source_options = load_parent_source()
    updated_list = []
    for eachfile in latest_updates:
        filepath = os.path.join(updated_specs_folder,eachfile)
        tmpdf = pd.read_csv(filepath, header=0)
        version = tmpdf.iloc[0]['typeversion']
        url = tmpdf.iloc[0]['url']
        tmpdict = parse_info_from_json(url,parent_source_options)
        updated_list.append(tmpdict)
    updated_df = pd.DataFrame(updated_list)
    return updated_df

In [142]:
def parse_version_from_url(url,classname):
    urlstrlist = url.split('/')
    bioschemasfile = urlstrlist[-1]
    no_ext = bioschemasfile.replace('.JSONLD','').replace('.jsonld','').replace('.JSON','').replace('.json','')
    version = no_ext.replace(classname+'_v','').replace(classname+'_','').replace('-type','').replace('-profile','')
    return version


def check_for_updates(spec_updated_df,original_df):
    update_needed = False
    if len(spec_updated_df)>0:
        update_needed = True
    if update_needed == True:
        original_url_list = original_df['url'].tolist()
        original_alias_list = [x.replace('blob','tree') for x in original_url_list]
        potentially_needs_updates = spec_updated_df.loc[~spec_updated_df['url'].isin(original_url_list)]
        if len(potentially_needs_updates) > 0:
            alias_check = potentially_needs_updates.loc[~potentially_needs_updates['url'].isin(original_alias_list)]
            if len(alias_check) > 0:
                update_needed = True
            else:
                update_needed = False
        else:
            update_needed = False
    return(update_needed)


def compare_versions(a,b):
    greaterlist = ['0.10','0.11','0.12','0.13','0.14','0.15']
    lesserlist = ['0.2','0.3','0.4','0.5','0.6','0.7','0.8','0.9']
    latest_version = None
    if a > b:
        latest_version = a
    else:
        for x in greaterlist:
            if x in a:
                weirdversion = True
                break
            else:
                weirdversion = False
        if weirdversion == False:
            latest_version = b
        else:
            for y in lesserlist:
                if y in b:
                    latest_version = a
                    break
                else:
                    latest_version = b    
    return latest_version


def update_spec_table(original_df,spec_updated_df):
    classes_to_update = spec_updated_df['name'].to_list()
    newdf = original_df.loc[~original_df['name'].isin(classes_to_update)]
    for eachclass in classes_to_update:
        updateversiondf = spec_updated_df.loc[spec_updated_df['name']==eachclass]
        updateversion = updateversiondf.iloc[0]['version']
        oldversiondf = original_df.loc[original_df['name']==eachclass]
        oldversion = oldversiondf.iloc[0]['version']
        latestversion = compare_versions(updateversion,oldversion)
        if latestversion == updateversion:
            newdf = pd.concat((newdf,updateversiondf),ignore_index=True)
        else :
            newdf = pd.concat((newdf,oldversiondf),ignore_index=True)
    return newdf

## To do:

1. Generate the updated_df
2. split out the updated_df to the individual tables that they should update
3. Check to see if the url is already in the table, if so, ignore it
 * get old table url.tolist generate a list of urls)
 * `newtable.loc[~newtable[url].isin(list of oldtable urls)]`
4. of the remaining entries, check if the name is already in the table
 * if it is, select the version with the higher number
 

In [117]:
script_path = ''
updated_df = generate_base_update_table(script_path)

## split the tables
deprecated = updated_df.loc[updated_df['version'].astype(str).str.contains('DEPRECATED')]
draftdf = updated_df.loc[updated_df['version'].astype(str).str.contains('DRAFT')]
releasedf = updated_df.loc[updated_df['version'].astype(str).str.contains('RELEASE')]
draft_profile = draftdf.loc[draftdf['type']=='Profile']
draft_type = draftdf.loc[draftdf['type']=='Type']

print(deprecated)

              namespace    name          subClassOf     type  \
0  bioschemasdeprecated  Beacon  schema:DataCatalog  Profile   

                           version  \
0  0.2-DRAFT-2018_04_23-DEPRECATED   

                                                 url  
0  https://github.com/BioSchemas/specifications/t...  


In [143]:
## check if a table needs updating

spec_updated_df = deprecated
original_df = pd.read_csv(os.path.join(script_path,'deprecated.txt'),delimiter='\t',header=0,index_col=0)
update_needed = check_for_updates(spec_updated_df,original_df)
newdf = update_spec_table(original_df,spec_updated_df)
print(newdf)


                name            subclassOf     type  \
0         DataRecord   schema:CreativeWork  Profile   
1  ProteinAnnotation  schema:BioChemEntity  Profile   
2             Beacon                   NaN  Profile   

                           version  \
0             0.2-DRAFT-2019_06_14   
1                        0.6-DRAFT   
2  0.2-DRAFT-2018_04_23-DEPRECATED   

                                                 url             namespace  \
0  https://github.com/BioSchemas/specifications/b...                   NaN   
1  https://github.com/BioSchemas/specifications/b...                   NaN   
2  https://github.com/BioSchemas/specifications/t...  bioschemasdeprecated   

           subClassOf  
0                 NaN  
1                 NaN  
2  schema:DataCatalog  


In [139]:
a = '0.10-DRAFT'
b = '0.8-DRAFT'
  
test = compare_versions(a,b)
print(test)

0.10-DRAFT


In [124]:
## if a table needs updating, select the files from the updatedf only if the version is greater
update_needed = True
if update_needed == True:



older is smaller
